In [175]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [176]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-02-03 07:21:59--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  78.8MB/s    in 0.3s    

2024-02-03 07:21:59 (78.8 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [177]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [178]:
# add your code here - consider creating a new cell for each section of code

In [179]:
df_merge = df_books.merge(df_ratings, how="inner", on="isbn")
df_merge = df_merge[["isbn", "user", "rating"]]

In [180]:
rm_isbn = df_merge.groupby(by=["isbn"]).agg({"rating": "count"}).query("rating < 100").index
rm_user = df_merge.groupby(by=["user"]).agg({"rating": "count"}).query("rating < 200").index

In [181]:
df_merge_drop = df_merge[~(df_merge.isbn.isin(rm_isbn) | df_merge.user.isin(rm_user))].dropna()

df_pivot = df_merge_drop.pivot_table(index="isbn", columns="user", values="rating").fillna(0)

In [182]:
model = NearestNeighbors(metric='cosine')

In [183]:
model.fit(df_pivot)

NearestNeighbors(metric='cosine')

In [184]:
book = "Where the Heart Is (Oprah's Book Club (Paperback))"

target_isbn = df_books[df_books.title.isin([book])].isbn
recomd_dist, recomd_idx = model.kneighbors(df_pivot.loc[target_isbn], 5)
recomd_isbn = df_pivot.index[recomd_idx.squeeze()[1:]].values.tolist()
recomd_title = df_books[df_books.isbn.isin(recomd_isbn)].title.values

recomd_df = pd.DataFrame({"title": recomd_title, "distance": recomd_dist.squeeze()[1:]}).sort_values("distance", ascending=False)
recommended_books = [book, recomd_df.values.tolist()]
recommended_books

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Surgeon', 0.7678344249725342],
  ['I Know This Much Is True', 0.7669050693511963],
  ['The Lovely Bones: A Novel', 0.7646420001983643],
  ['The Weight of Water', 0.715856671333313]]]

In [185]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  target_isbn = df_books[df_books.title.isin([book])].isbn
  recomd_dist, recomd_idx = model.kneighbors(df_pivot.loc[target_isbn], 5)
  recomd_isbn = df_pivot.index[recomd_idx.squeeze()[1:]].values.tolist()
  recomd_title = df_books[df_books.isbn.isin(recomd_isbn)].title.values
  recomd_df = pd.DataFrame({"title": recomd_title, "distance": recomd_dist.squeeze()[1:]}).sort_values("distance", ascending=False)
  recommended_books = [book, recomd_df.values.tolist()]

  return recommended_books

In [186]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Surgeon', 0.7678344249725342], ['I Know This Much Is True', 0.7669050693511963], ['The Lovely Bones: A Novel', 0.7646420001983643], ['The Weight of Water', 0.715856671333313]]]
You passed the challenge! 🎉🎉🎉🎉🎉
